In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import openai

load_dotenv(override=True)
openai.api_key=os.getenv('OPENAI_API_KEY') 
if not openai.api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set.")


header={"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"}
class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url,headers=header)
        soup=BeautifulSoup(response.content,'html.parser')

        self.title=soup.title.string if soup.title else "No title tag"
        for irrelevant in soup.body(["style","script","img"]):
            irrelevant.decompose()

        self.text=soup.body.get_text(separator="\n",strip=True)

llm=Website("https://edwarddonner.com")

system_prompt="i need your help in categorising thge website of the industrial sector basing on its sectors and all its involved in "

def user_prompt_for(llm):
    user_prompt=f"the website generated has a title of {llm.title}\n"                                                 
    user_prompt="retrieve all the necessary information creating a summary of the important notes \
                with all the necessary contact information of each summarised sector \
                an easy means of how to make a events in calendar \n"
    user_prompt +=llm.text
    return user_prompt

def message_for(llm):
    message=[{"role":"user"  ,"content":user_prompt_for(llm) },
              {"role":"system" ,"content":system_prompt }]
    return message

def summarize(url):
    llm=Website("https://edwarddonner.com")
    response=openai.chat.completions.create(
        model="gpt-4o-mini",
        messages= message_for(llm)
    )
    return response.choices[0].message.content                                 

def display_summary(url):
    summary=summarize(url)
    display(Markdown(summary))
    return display
display_summary("https://edwarddonner.com")

        